# Bio-Argo match-ups
**Author:** Eli Holmes (NOAA)</br>
Last updated: November 21, 2025

We have upwards of 33k bbp700 and 10k chl-a points for the global bio-Argo data. The goal is to get the Rrs data, all bands, for each of these. However, with so many points we need to be careful with memory. Just opening the meta-data for all the Rrs files, 500+ days since March 2024, will eat up a lot of memory.

**What do we do?**

1. When we open the PACE Rrs files, we create `ds` with `chunks{}` so that dask knows the chunk structure.
2. We compute the vector of indices for the Argo points and use `ds.vindex([vector_i, vector_j)`. That will load only the needed chunks.
3. We work through the files in batches and use a dask cluster that we close after finishing with a batch. This will keep the memory from exploding on us as we open files.

**Dask cluster workflow**

1. Create a function to process one file
2. Create a batch for loop that does
    * Grab a batch of file urls (to their location in the cloud)
    * Open a dask cluster
    * Process the files with the process file function
    * Close the cluster when done
    * Grab the next batch of file urls
    * Repeat


## Load the Argo data

In [18]:
# Load data from GitHub
df_y_name="CHLA"; df_lat_name="LATITUDE"; df_lon_name="LONGITUDE"; df_time_name="TIME"
import pandas as pd
base_url="https://raw.githubusercontent.com/fish-pace/2025-tutorials/main/tutorial_data/"
url = f"{base_url}argo_bgc_global_surface_{df_y_name}.parquet"
df = pd.read_parquet(url)

## Get the file urls for the PACE Data

In [19]:
import earthaccess
auth = earthaccess.login()
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)
import xarray as xr
results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_RRS",
    temporal = (df.TIME.min(), df.TIME.max()),
    granule_name="*.DAY.*.4km.nc"
)
# all days since Mar 2024; pqdm args to quiet output
fileset = earthaccess.open(results, pqdm_kwargs={"disable": True} );

Check the format of individuals day data.

In [20]:
import xarray as xr
ds = xr.open_dataset(
        fileset[0],
        chunks={}
    )
ds

<xarray.Dataset> Size: 26GB
Dimensions:     (wavelength: 172, lat: 4320, lon: 8640, rgb: 3,
                 eightbitcolor: 256)
Coordinates:
  * wavelength  (wavelength) float64 1kB 346.0 348.0 351.0 ... 714.0 717.0 719.0
  * lat         (lat) float32 17kB 89.98 89.94 89.9 ... -89.9 -89.94 -89.98
  * lon         (lon) float32 35kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    Rrs         (lat, lon, wavelength) float32 26GB dask.array<chunksize=(16, 1024, 8), meta=np.ndarray>
    palette     (rgb, eightbitcolor) uint8 768B dask.array<chunksize=(3, 256), meta=np.ndarray>
Attributes: (12/64)
    product_name:                      PACE_OCI.20240305.L3m.DAY.RRS.V3_1.Rrs...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    identifier_product_doi:            10.5067/PACE/OCI/L3M/RRS/3.1
    keywords:                          Earth Science > Oceans > Ocean Optics ...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         3861548
    data_minimum:                      -0.009998
    data_maximum:                      0.092371605

# Workflow to process one file

Assume:

* The files are for a regular grid, i.e. level 3 data
* The files do not have a time coordinate. This is how the PACE level 3 daily and 8-day files are currently structured.
* We are looking at a variable, like Rrs, that has a 3rd dimension of wavelength or similar. Usually called wavelength or wavelength_3d. So the dims are (lat, lon, 3rd dim)
* `time_coverage_start` and `time_coverage_end` exist as attributes in the files.
* `ds[ds_vec_name]` is a 1D coordinate aligned with the last dimension of ds_var_name.

Process Steps:

1.  Subset our Argo dataframe to only those rows within in the  time window covered by the cloud-hosted xarray dataset (`ds`). Let's call that `df_day`
2.  Get the lat/lon indices of `ds` that correspond to the lat/lon rows in `df_day`. These indices will look like `[0, 3, 6]` and `[0, 10, 15]` corresponding to index pairs, like `(3, 10)` corresponding to the lat/lon pair in a row of `df_day`.
3.  Do a fast point retrieval from `ds`. `ds` is cloud optimized (chunked) file and we only want to the chunks that we need. We do not need to load all the chunks. I/O of cloud-hosted data is the slow part so don't touch more of the data than you need to.
4. Build a dataframe with the predictor variables that we want from `ds`.

In [21]:
# New process one file code
# need to create a function that has only the thing will will iterate over.
# everything else will be a variable in local memory
# Load data from GitHub
import pandas as pd
import earthaccess
import xarray as xr

# specify the ds colnames and variable we want (Rrs)
ds_lat_name="lat"; ds_lon_name="lon"; ds_time_name="time"; ds_vec_name="wavelength"; ds_var_name="Rrs"; 

def process_one_file(record):
    # Make sure we're logged in; earthaccess will refresh token as needed
    earthaccess.login()  # if you're using netrc/env, this is cheap

    # If record is a UMM dict from search_data:
    files = earthaccess.open([record], pqdm_kwargs={"disable": True} )   # returns a list of file-like objects
    f = files[0]

    with xr.open_dataset(f, chunks={}, cache=False) as ds:

        # --- Step 1: subset df to the time window in ds
        t_start = pd.to_datetime(ds.attrs["time_coverage_start"], utc=True)
        t_end   = pd.to_datetime(ds.attrs["time_coverage_end"], utc=True)
        df_times = pd.to_datetime(df[df_time_name], utc=True)   
        df_day = df[(df_times >= t_start) & (df_times < t_end)]

        # Stop if no days in df match day in ds
        if df_day.empty: return None

        # --- Step 2: Get an array of Rrs values for the lat/lon in df_day
        # will need index and values later
        lat_idx = ds.get_index(ds_lat_name)
        lon_idx = ds.get_index(ds_lon_name)
        lat_vals = ds[ds_lat_name].values
        lon_vals = ds[ds_lon_name].values
        # Get the lat/lon vals in df_day
        df_lats = df_day[df_lat_name].to_numpy(dtype=float)
        df_lons = df_day[df_lon_name].to_numpy(dtype=float)
        # Use pandas indexer to quickly find indices for vals nearest df
        lat_i = lat_idx.get_indexer(df_lats, method="nearest")
        lon_i = lon_idx.get_indexer(df_lons, method="nearest")

        # Each day has only 10-30 Argo points. This will be faster than .vindex
        def sample_rrs_points(ds, lats, lons, var_name="Rrs"):
            import numpy as np
            R = ds[var_name]  # (lat, lon, wavelength)
            spectra = [
                R.sel(lat=i, lon=j, method="nearest").values
                for i, j in zip(lats, lons)
            ]
            return np.stack(spectra, axis=0)
        
        var_vals=sample_rrs_points(ds, df_lats, df_lons)    
        
        # --- Step 4: build a dataframe with our Argo points and PACE data ---
        data = {
            ds_time_name: pd.to_datetime(df_day[df_time_name], utc=True),
            ds_lat_name:  lat_vals[lat_i],
            ds_lon_name:  lon_vals[lon_i],
            "y": df_day[df_y_name].to_numpy(),
            "df_lat": df_lats,
            "df_lon": df_lons
            }

        # Get the wavelength values for our col names
        vec_vals = ds[ds_vec_name].values
        for j, v in enumerate(vec_vals):
            label = int(v) # make integer for nicer label
            col_name = f"{ds_var_name}_{label}"
            data[col_name] = var_vals[:, j].astype(float)

        return pd.DataFrame(data)

In [22]:
%%time
# test that it works
res=process_one_file(results[1])
res.head()

CPU times: user 5.81 s, sys: 2.12 s, total: 7.93 s
Wall time: 27.2 s


,time,lat,lon,y,df_lat,df_lon,Rrs_346,Rrs_348,Rrs_351,Rrs_353,...,Rrs_706,Rrs_707,Rrs_708,Rrs_709,Rrs_711,Rrs_712,Rrs_713,Rrs_714,Rrs_717,Rrs_719
19,2024-03-06 07:12:00+00:00,-68.020836,8.520839,0.453600,-68.040133,8.532055,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,2024-03-06 06:23:53.028999936+00:00,-51.520836,89.479172,0.357700,-51.510311,89.460415,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2024-03-06 21:47:00+00:00,-57.687504,157.062515,0.114950,-57.682947,157.049877,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2024-03-06 12:43:12.000999936+00:00,-53.020836,166.187515,0.569108,-53.022200,166.197400,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,2024-03-07 01:09:49+00:00,-37.104172,-149.187500,0.103240,-37.091700,-149.182900,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Run through the PACE files in batches

Just in case something hangs.

### Create a function to run a batch

* create cluster
* run `process_one_file` using `.map` over all file urls
* gather results (pandas dataframe)
* close cluster and return results

In [23]:
# run_batch function
import pandas as pd
def run_batch(batch_results):
    results = []
    for k, res in enumerate(batch_results, start=1):
        res = process_one_file(res)
        results.append(res)
    return pd.concat(results, ignore_index=True)

In [24]:
%%time
# test that it runs
run_batch(results[0:1])

CPU times: user 3.31 s, sys: 1.11 s, total: 4.42 s
Wall time: 17.5 s


,time,lat,lon,y,df_lat,df_lon,Rrs_346,Rrs_348,Rrs_351,Rrs_353,...,Rrs_706,Rrs_707,Rrs_708,Rrs_709,Rrs_711,Rrs_712,Rrs_713,Rrs_714,Rrs_717,Rrs_719
0,2024-03-05 11:39:00+00:00,-45.770836,136.562515,0.727765,-45.756765,136.574703,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-03-05 06:58:45.000999936+00:00,-34.145836,-165.729172,0.023080,-34.152200,-165.736500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-03-06 00:25:00+00:00,-31.895838,37.145840,0.018225,-31.877871,37.132439,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-03-06 01:11:24+00:00,-10.687503,49.729172,0.202350,-10.701914,49.728228,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-03-05 08:16:05.000999936+00:00,0.687497,-165.437500,0.514266,0.673300,-165.453300,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-03-05 15:35:00+00:00,39.145832,-66.562500,0.317225,39.151501,-66.574600,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-03-05 01:16:05+00:00,29.062498,-17.229162,0.881961,29.082600,-17.213600,0.006416,0.0069,0.007244,0.007086,...,0.000218,0.000218,0.000224,0.000214,0.000214,0.000196,0.000174,0.000166,0.00016,0.000214
7,2024-03-05 02:08:10+00:00,25.104164,148.145844,0.036300,25.111515,148.146488,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2024-03-05 11:37:00+00:00,68.062500,-5.104161,0.036000,68.045173,-5.118235,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-03-05 11:32:39.019000064+00:00,69.687500,4.187506,0.021900,69.673916,4.204859,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## For loop over our batches

* get urls for a batch of 20 PACE files in cloud
* save results to a file in case the process crashes
* later we will reconstruct from the files. The files are small.

## CHLA

In [ ]:
from pathlib import Path
import pandas as pd

results_dir = Path("data/matchups")
results_dir.mkdir(exist_ok=True)

all_batches = []
batch_size = 10
var = "chla"

for batch_idx, i in enumerate(range(0, len(fileset), batch_size), start=1):
    batch = fileset[i:i+batch_size]
    print(f"Batch {batch_idx}: {len(batch)} files")

    # If this batch was already done, skip it
    batch_path = results_dir / f"{var}_matchups_batch_{batch_idx:03d}.parquet"
    if batch_path.exists():
        print(f"  -> Skipping batch {batch_idx}, found {batch_path}")
        continue

    if batch_idx % 10:
        fileset = refresh_token()

    df_batch = run_batch(batch)  # returns a DataFrame

    # Save this batch immediately
    df_batch.to_parquet(batch_path, index=False)
    print(f"  -> Saved {len(df_batch)} rows to {batch_path}")


In [12]:
# When everything is done, you can merge all batches:
batch_files = sorted(results_dir.glob(f"{var}_matchups_batch_*.parquet"))
dfs = [pd.read_parquet(f) for f in batch_files]
df_all = pd.concat(dfs, ignore_index=True)
df_all.to_parquet(f"tutorial_data/{var}_argo_rss_all.parquet", index=False)
rrs_cols = [c for c in df_all.columns if c.startswith("Rrs_")]
df_all_clean = df_all.dropna(subset=rrs_cols, how="all").reset_index(drop=True)
df_all_clean.to_parquet(f"tutorial_data/{var}_argo_rss.parquet", index=False)
print(f"Total rows: {len(df_all_clean)}")


Total rows: 12140


In [13]:
print(f"Total rows: {len(df_all_clean)}")

Total rows: 2271


## BBP700

In [25]:
# BBP700
# Load data from GitHub
df_y_name="BBP700"; df_lat_name="LATITUDE"; df_lon_name="LONGITUDE"; df_time_name="TIME"
import pandas as pd
base_url="https://raw.githubusercontent.com/fish-pace/2025-tutorials/main/tutorial_data/"
url = f"{base_url}argo_bgc_global_surface_{df_y_name}.parquet"
df = pd.read_parquet(url)

In [26]:
# BBP700
from pathlib import Path
import pandas as pd

results_dir = Path("data/matchups")
results_dir.mkdir(exist_ok=True)

all_batches = []
batch_size = 10
var = "bbp700"

for batch_idx, i in enumerate(range(0, len(results), batch_size), start=1):
    # If this batch was already done, skip it
    batch_path = results_dir / f"{var}_matchups_batch_{batch_idx:03d}.parquet"
    if batch_path.exists():
        print(f"  -> Skipping batch {batch_idx}, found {batch_path}")
        continue

    batch = results[i:i+batch_size]
    print(f"Batch {batch_idx}: {len(batch)} files")
        
    df_batch = run_batch(batch)  # returns a DataFrame

    # Save this batch immediately
    df_batch.to_parquet(batch_path, index=False)
    print(f"  -> Saved {len(df_batch)} rows to {batch_path}")


  -> Skipping batch 1, found data/matchups/bbp700_matchups_batch_001.parquet
  -> Skipping batch 2, found data/matchups/bbp700_matchups_batch_002.parquet
  -> Skipping batch 3, found data/matchups/bbp700_matchups_batch_003.parquet
  -> Skipping batch 4, found data/matchups/bbp700_matchups_batch_004.parquet
  -> Skipping batch 5, found data/matchups/bbp700_matchups_batch_005.parquet
  -> Skipping batch 6, found data/matchups/bbp700_matchups_batch_006.parquet
  -> Skipping batch 7, found data/matchups/bbp700_matchups_batch_007.parquet
  -> Skipping batch 8, found data/matchups/bbp700_matchups_batch_008.parquet
  -> Skipping batch 9, found data/matchups/bbp700_matchups_batch_009.parquet
  -> Skipping batch 10, found data/matchups/bbp700_matchups_batch_010.parquet
  -> Skipping batch 11, found data/matchups/bbp700_matchups_batch_011.parquet
  -> Skipping batch 12, found data/matchups/bbp700_matchups_batch_012.parquet
  -> Skipping batch 13, found data/matchups/bbp700_matchups_batch_013.par

In [4]:
# When everything is done, you can merge all batches:
from pathlib import Path
import pandas as pd

var = "bbp700"
results_dir = Path("data/matchups")

batch_files = sorted(results_dir.glob(f"{var}_matchups_batch_*.parquet"))
dfs = [pd.read_parquet(f) for f in batch_files]
df_all = pd.concat(dfs, ignore_index=True)
df_all.to_parquet(f"tutorial_data/{var}_argo_rss_all.parquet", index=False)
rrs_cols = [c for c in df_all.columns if c.startswith("Rrs_")]
df_all_clean = df_all.dropna(subset=rrs_cols, how="all").reset_index(drop=True)
df_all_clean.to_parquet(f"tutorial_data/{var}_argo_rss.parquet", index=False)

Total rows: 5599


In [5]:
print(f"Total rows: {len(df_all_clean)}")
print(f"Total rows: {len(df_all)}")


Total rows: 5599
Total rows: 29950


---------------------
## Testing stuff